In [ ]:
%load_ext rpy2.ipython

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, classification_report

# Use the combined data that was successfully generated in a previous step
# This dataframe contains the expression data for significant genes and the corresponding labels
# combined_data_for_ml was created from exprData_gse34526_sig and normalized_counts_gse137684_sig
# and already has samples as rows and genes as columns, with a 'label' column.
# We will use this data directly for the machine learning pipeline.

# Check if the dataframe exists and is not empty
if 'combined_data_for_ml' in locals() and not combined_data_for_ml.empty:
    print("Using existing 'combined_data_for_ml' for machine learning.")
    data = combined_data_for_ml
else:
    print("Error: 'combined_data_for_ml' not found or is empty. Cannot proceed with ML.")
    # You might want to add a step here to load the data from the CSV if it was saved
    # For example:
    # try:
    #     data = pd.read_csv("degs_expression_matrix.csv")
    #     if 'Unnamed: 0' in data.columns:
    #         data = data.drop('Unnamed: 0', axis=1)
    #     print("Loaded data from 'degs_expression_matrix.csv' for machine learning.")
    # except FileNotFoundError:
    #     print("Error: 'degs_expression_matrix.csv' not found. Cannot proceed with ML.")
    #     exit() # Exit the cell if data cannot be loaded


    # If you want to retry the R processing to generate the data, you would call the R cell here
    # However, given the persistent parsing issues, using the existing data is a better approach for now.
    raise ValueError("Required data 'combined_data_for_ml' is not available.")


X = data.drop("label", axis=1)  # features
y = data["label"]  # 0 = Control, 1 = PCOS

# Train-test split (still useful for a final evaluation, but cross-validation is better for model selection)
# Using stratify for balanced splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Initialize models
lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
svm = SVC(kernel='linear', probability=True)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Perform cross-validation
# Using StratifiedKFold to ensure folds have representative proportions of classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # Using 5 splits for cross-validation

print("Cross-validation AUC scores:")

# LASSO Logistic Regression Cross-validation
lasso_cv_scores = cross_val_score(lasso, X, y, cv=cv, scoring='roc_auc')
print(f"LASSO (CV) AUC: Mean={np.mean(lasso_cv_scores):.4f}, Std={np.std(lasso_cv_scores):.4f}")

# Support Vector Machine Cross-validation
svm_cv_scores = cross_val_score(svm, X, y, cv=cv, scoring='roc_auc')
print(f"SVM (CV) AUC: Mean={np.mean(svm_cv_scores):.4f}, Std={np.std(svm_cv_scores):.4f}")

# XGBoost Cross-validation
xgb_cv_scores = cross_val_score(xgb, X, y, cv=cv, scoring='roc_auc')
print(f"XGBoost (CV) AUC: Mean={np.mean(xgb_cv_scores):.4f}, Std={np.std(xgb_cv_scores):.4f}")

# Optional: You can still evaluate on the test set for comparison
# print("\nTest set AUC scores:")
# print("LASSO AUC:", roc_auc_score(y_test, lasso.predict_proba(X_test)[:,1]))
# print("SVM AUC:", roc_auc_score(y_test, svm.predict_proba(X_test)[:,1]))
# print("XGBoost AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))

# Further steps to address overfitting and model interpretation can follow,
# such as feature selection based on cross-validation, or model interpretation
# techniques (e.g., examining LASSO coefficients or XGBoost feature importances).

Using existing 'combined_data_for_ml' for machine learning.
Cross-validation AUC scores:
LASSO (CV) AUC: Mean=1.0000, Std=0.0000
SVM (CV) AUC: Mean=1.0000, Std=0.0000


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:22:03] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:22:08] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:22:12] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:22:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:22:19] WARNING: /w

XGBoost (CV) AUC: Mean=0.9500, Std=0.1000


In [ ]:
%%R
# Differential gene expression analysis for GSE34526
print("Analyzing GSE34526 with DESeq2:")

# Download the GSE34526 GEO object to get sample information and expression data
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data (count data) and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Ensure the expression data is in integer mode as required by DESeq2
# This dataset might be normalized or processed, so we might need to adjust based on its nature.
# Based on typical GEO data for microarrays, this might not be raw counts.
# If it's not counts, DESeq2 is not the appropriate tool. We might need to use limma.
# Let's first check the data class and consider using limma if it's not integer counts.
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Assuming it's count data (if not, we'll need to switch to limma)
# Identify the condition column from phenoData
print("\nColumn names of GSE34526 phenoData:")
print(colnames(phenoData_gse34526))
print("\nFirst few rows of GSE34526 phenoData:")
print(head(phenoData_gse34526))

# Based on the phenoData, identify the appropriate column for condition.
# Let's assume 'source_name_ch1' or 'characteristics_ch1' contains condition information.
# You might need to adjust this based on the actual data.
# For demonstration, let's assume 'source_name_ch1' has the condition.
# If it does not, you will need to manually identify the correct column.
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  # Clean up level names when creating the factor
  condition_factor <- factor(phenoData_gse34526$'characteristics_ch1.1')
  levels(condition_factor) <- make.names(levels(condition_factor))
  colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                                 condition = condition_factor)
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1' if 'characteristics_ch1.1' is not available
   # Clean up level names when creating the factor
  condition_factor <- factor(phenoData_gse34526$'source_name_ch1')
  levels(condition_factor) <- make.names(levels(condition_factor))
   colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                                 condition = condition_factor)
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1' if 'source_name_ch1' is not available
   # Clean up level names when creating the factor
  condition_factor <- factor(phenoData_gse34526$'characteristics_ch1')
  levels(condition_factor) <- make.names(levels(condition_factor))
   colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                                 condition = condition_factor)
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Ensure row names of colData match column names of exprData
colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]


# Check if the expression data is suitable for DESeq2 (i.e., integer counts)
if (is.integer(exprData_gse34526) || all(exprData_gse34526 == floor(exprData_gse34526))) {
  print("GSE34526 expression data is integer counts. Using DESeq2 for analysis.")
  # Create a DESeqDataSet object for GSE34526
  dds_gse34526 <- DESeqDataSetFromMatrix(countData = round(exprData_gse34526), # Round to ensure integers
                                         colData = colData_gse34526,
                                         design = ~condition)

  # Run DESeq2 analysis
  # Check if standard dispersion estimation fails and apply workaround if needed
  tryCatch({
    dds_gse34526 <- DESeq(dds_gse34526)
  }, error = function(e) {
    if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
      message("Standard dispersion estimation failed. Using gene-wise estimates as final estimates.")
      dds_gse34526 <<- estimateSizeFactors(dds_gse34526)
      dds_gse34526 <<- estimateDispersionsGeneEst(dds_gse34526)
      dispersions(dds_gse34526) <<- mcols(dds_gse34526)$dispGeneEst
      dds_gse34526 <<- nbinomWaldTest(dds_gse34526)
    } else {
      stop(e) # Re-throw the error if it's not the expected one
    }
  })

  # Get differential expression results
  if (inherits(dds_gse34526, "DESeqDataSet")) {
      condition_levels <- levels(colData_gse34526$condition)
      if (length(condition_levels) >= 2) {
          res_gse34526 <- results(dds_gse34526, contrast=c("condition", condition_levels[2], condition_levels[1]))
          # Display summary of results
          print(summary(res_gse34526))
          # Display top genes
          print(head(res_gse34526[order(res_gse34526$padj), ]))
      } else {
          print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis.")
      }
  } else {
      print("Error: DESeq2 analysis failed for GSE34526. dds_gse34526 is not a valid DESeqDataSet object.")
  }

} else {
  print("GSE34526 expression data is not integer counts. Using limma for analysis.")
  # Load limma library
  library(limma)

  # Create a design matrix for limma
  design <- model.matrix(~condition, data = colData_gse34526)

  # Print column names of the design matrix to help with contrast specification
  print("\nColumn names of the design matrix for GSE34526:")
  print(colnames(design))

  # Fit the linear model
  fit <- lmFit(exprData_gse34526, design)

  # Apply empirical Bayes moderation
  fit <- eBayes(fit)

  # Get differential expression results
  # Assuming the contrast is between the second and first level of the condition factor
  if (length(levels(colData_gse34526$condition)) >= 2) {
      # Clean up level names for contrast
      clean_levels <- levels(colData_gse34526$condition) # Levels are already cleaned when creating colData
      # Use the column names of the design matrix to specify the contrast
      contrast_matrix <- makeContrasts(contrasts=paste(colnames(design)[2], "-", colnames(design)[1], sep=""), levels=design)
      fit2 <- contrasts.fit(fit, contrast_matrix)
      fit2 <- eBayes(fit2)
      res_gse34526 <- topTable(fit2, number=Inf, adjust.method="BH")

      # Display summary of results (limma)
      print(summary(res_gse34526))
      # Display top genes (limma)
      print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))

  } else {
      print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
  }
}

[1] "Analyzing GSE34526 with DESeq2:"
[1] "Class of expression data for GSE34526: matrix"
[2] "Class of expression data for GSE34526: array" 
[1] "\nColumn names of GSE34526 phenoData:"
 [1] "title"                   "geo_accession"          
 [3] "status"                  "submission_date"        
 [5] "last_update_date"        "type"                   
 [7] "channel_count"           "source_name_ch1"        
 [9] "organism_ch1"            "characteristics_ch1"    
[11] "characteristics_ch1.1"   "treatment_protocol_ch1" 
[13] "growth_protocol_ch1"     "molecule_ch1"           
[15] "extract_protocol_ch1"    "label_ch1"              
[17] "label_protocol_ch1"      "taxid_ch1"              
[19] "hyb_protocol"            "scan_protocol"          
[21] "description"             "data_processing"        
[23] "platform_id"             "contact_name"           
[25] "contact_email"           "contact_laboratory"     
[27] "contact_department"      "contact_institute"      
[29] "contact_ad

Found 1 file(s)
GSE34526_series_matrix.txt.gz
Using locally cached version: /tmp/Rtmp5srNh3/GSE34526_series_matrix.txt.gz
Using locally cached version of GPL570 found here:
/tmp/Rtmp5srNh3/GPL570.soft.gz 
In addition: Warning message:
In makeContrasts(contrasts = paste(colnames(design)[2], "-", colnames(design)[1],  :
  Renaming (Intercept) to Intercept


In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("GEOquery")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 'GEOquery'
also installing the dependencies ‘statmod’, ‘XML’, ‘R.oo’, ‘R.methodsS3’, ‘limma’, ‘rentrez’, ‘R.utils’

trying URL 'https://cran.rstudio.com/src/contrib/statmod_1.5.0.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/XML_3.99-0.19.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/R.oo_1.27.1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/R.methodsS3_1.8.2.tar.gz'
trying URL 'https://bioconductor.org/packages/3.21/bioc/src/contrib/limma_3.64.3.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/rentrez_1.2.4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/R.utils_2.13.0.tar.gz'
trying URL 'https://bioconductor.org/packages/3.21/bioc/src/contrib/GEOquery_2.76.

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("DESeq2")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.26.tar.gz'
Content type 'application/x-gzip' length 594489 bytes (580 KB)
downloaded 580 KB


The downloaded source packages are in
	‘/tmp/Rtmp5srNh3/downloaded_packages’
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 'BiocVersion', 'DESeq2'
also installing the dependencies ‘formatR’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘SparseArray’, ‘lambda.r’, ‘futile.options’, ‘GenomeInfoDb’, ‘XVector’, ‘S4Arrays’, ‘DelayedArray’, ‘futile.logger’, ‘snow’, ‘BH’, ‘S4Vectors’, ‘IRanges’, ‘GenomicRanges’, ‘SummarizedExperiment’, ‘BiocGenerics’, ‘Biobase’, ‘BiocParallel’, ‘matrixStats’, ‘locfit’, ‘MatrixGenerics’, ‘Rcp

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(DESeq2)
library(utils) # For read.table
library(GEOquery)

# Download supplementary files for GSE34526
print("Downloading supplementary files for GSE34526...")
getGEOSuppFiles("GSE34526")

# Download supplementary files for GSE137684
print("\nDownloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# List files in the downloaded directories to check for count data files
print("\nListing files in GSE34526 directory:")
list.files("GSE34526/")

print("\nListing files in GSE137684 directory:")
list.files("GSE137684/")

# Extract the tar file for GSE137684
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# List files in the extracted directory
print("\nListing files in GSE137684_extracted directory:")
list.files("GSE137684_extracted/")

# Define the directory containing the extracted count files for GSE137684
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
# Assuming the files have gene IDs in a specific column and counts in another
# You might need to adjust col.names and the column index based on the actual file format
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, it seems the data starts after row 6.
  # Column V1 is likely the gene ID and V11 is the count data.
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE) # Adjust skip as needed
  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  data.frame(gene_id = data$V1, count = as.integer(data$V11)) # Convert counts to integer
})

# Combine the data frames into a single matrix
# Assuming all files have the same gene order
count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))

# Replace NA values with 0
count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0

# Set row names to gene IDs
rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id

# Set column names to sample names (e.g., GSM numbers)
colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$"))

# Download the GSE137684 GEO object to get sample information
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Display column names and first few rows of phenoData to help identify the correct condition column
print("\nColumn names of GSE137684 phenoData:")
print(colnames(phenoData_gse137684))
print("\nFirst few rows of GSE137684 phenoData:")
print(head(phenoData_gse137684))

# Create a colData dataframe with sample information for GSE137684
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = colnames(count_matrix_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]


# Create a DESeqDataSet object for GSE137684
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column in your design formula

# Run DESeq2 analysis
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})


# Get differential expression results
# Check if dds_gse137684 is a DESeqDataSet object before getting results
if (inherits(dds_gse137684, "DESeqDataSet")) {
  # Replace 'contrast' with your actual contrast based on your design formula
  # Example: results(dds_gse137684, contrast=c("condition","treatment","control"))
  # Get the levels of the condition factor to define the contrast
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    res_gse137684 <- results(dds_gse137684, contrast=c("condition", condition_levels[2], condition_levels[1])) # Assuming the first level is the reference
    # Display summary of results
    print(summary(res_gse137684))

    # Display top genes
    print(head(res_gse137684[order(res_gse137684$padj), ]))
  } else {
    print("Error: Not enough levels in the condition variable to perform differential expression analysis.")
  }
} else {
  print("Error: DESeq2 analysis failed. dds_gse137684 is not a valid DESeqDataSet object.")
}

[1] "Downloading supplementary files for GSE34526..."
[1] "\nDownloading supplementary files for GSE137684..."
[1] "\nListing files in GSE34526 directory:"
[1] "\nListing files in GSE137684 directory:"
[1] "Extracting GSE137684_RAW.tar..."
[1] "\nListing files in GSE137684_extracted directory:"
[1] "\nColumn names of GSE137684 phenoData:"
 [1] "title"                   "geo_accession"          
 [3] "status"                  "submission_date"        
 [5] "last_update_date"        "type"                   
 [7] "channel_count"           "source_name_ch1"        
 [9] "organism_ch1"            "characteristics_ch1"    
[11] "characteristics_ch1.1"   "characteristics_ch1.2"  
[13] "treatment_protocol_ch1"  "molecule_ch1"           
[15] "extract_protocol_ch1"    "label_ch1"              
[17] "label_protocol_ch1"      "taxid_ch1"              
[19] "hyb_protocol"            "scan_protocol"          
[21] "data_processing"         "data_processing.1"      
[23] "platform_id"             "

Using locally cached version of supplementary file(s) GSE34526 found here:
/content/GSE34526/GSE34526_RAW.tar 
Using locally cached version of supplementary file(s) GSE137684 found here:
/content/GSE137684/GSE137684_RAW.tar 
Found 1 file(s)
GSE137684_series_matrix.txt.gz
Using locally cached version: /tmp/Rtmp5srNh3/GSE137684_series_matrix.txt.gz
Using locally cached version of GPL17077 found here:
/tmp/Rtmp5srNh3/GPL17077.soft.gz 
converting counts to integer mode
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  for column names in R. [This is a message, not a warning or an error]
  Note: levels of factors in the design contain characters other than
  letters, numbers, '_' and '.'. It is recommended (but not required) to use
  only letters, numbers, and delimiters '_' or '.', as these are safe characters
  

In [ ]:
%%R
library(DESeq2)
library(utils) # For read.table

# Define the directory containing the extracted count files for GSE137684
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
# Assuming the files have gene IDs in a specific column and counts in another
# You might need to adjust col.names and the column index based on the actual file format
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, it seems the data starts after row 6.
  # Column V1 is likely the gene ID and V11 is the count data.
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE) # Adjust skip as needed
  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  data.frame(gene_id = data$V1, count = as.integer(data$V11)) # Convert counts to integer
})

# Combine the data frames into a single matrix
# Assuming all files have the same gene order
count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))

# Set row names to gene IDs
rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id

# Set column names to sample names (e.g., GSM numbers)
colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$"))

# Create a colData dataframe with sample information for GSE137684
# You need to replace this with actual experimental design information for GSE137684.
# For now, using a placeholder colData with just sample names and a 'condition' column.
colData_gse137684 <- data.frame(row.names = colnames(count_matrix_gse137684),
                                condition = factor(rep("case", ncol(count_matrix_gse137684)))) # Replace with actual conditions

# Create a DESeqDataSet object for GSE137684
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Replace ~condition with your actual design formula

# Run DESeq2 analysis
dds_gse137684 <- DESeq(dds_gse137684)

# Get differential expression results
# Replace 'contrast' with your actual contrast based on your design formula
# Example: results(dds_gse137684, contrast=c("condition","treatment","control"))
res_gse137684 <- results(dds_gse137684)

# Display summary of results
print(summary(res_gse137684))

# Display top genes
print(head(res_gse137684[order(res_gse137684$padj), ]))

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R
# Extract the tar file for GSE137684
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# List files in the extracted directory
print("\nListing files in GSE137684_extracted directory:")
list.files("GSE137684_extracted/")

[1] "Extracting GSE137684_RAW.tar..."
[1] "\nListing files in GSE137684_extracted directory:"
 [1] "GSM4084711_1.txt.gz"  "GSM4084712_3.txt.gz"  "GSM4084713_5.txt.gz" 
 [4] "GSM4084714_9.txt.gz"  "GSM4084715_10.txt.gz" "GSM4084716_12.txt.gz"
 [7] "GSM4084717_13.txt.gz" "GSM4084718_14.txt.gz" "GSM4084719_15.txt.gz"
[10] "GSM4084720_16.txt.gz" "GSM4084721_17.txt.gz" "GSM4084722_18.txt.gz"


In [ ]:
%%R
# Download supplementary files for GSE34526
print("Downloading supplementary files for GSE34526...")
getGEOSuppFiles("GSE34526")

# Download supplementary files for GSE137684
print("\nDownloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# List files in the downloaded directories to check for count data files
print("\nListing files in GSE34526 directory:")
list.files("GSE34526/")

print("\nListing files in GSE137684 directory:")
list.files("GSE137684/")

[1] "Downloading supplementary files for GSE34526..."
[1] "\nDownloading supplementary files for GSE137684..."
[1] "\nListing files in GSE34526 directory:"
[1] "\nListing files in GSE137684 directory:"
[1] "GSE137684_RAW.tar"


trying URL 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE34nnn/GSE34526/suppl//GSE34526_RAW.tar?tool=geoquery'
Content type 'application/x-tar' length 43960320 bytes (41.9 MB)
downloaded 41.9 MB

trying URL 'https://ftp.ncbi.nlm.nih.gov/geo/series/GSE137nnn/GSE137684/suppl//GSE137684_RAW.tar?tool=geoquery'
Content type 'application/x-tar' length 154163200 bytes (147.0 MB)
downloaded 147.0 MB



In [ ]:
%%R
library(GEOquery)

# Download the datasets from GEO
gse34526 <- getGEO("GSE34526", GSEMatrix = TRUE)
gse137684 <- getGEO("GSE137684", GSEMatrix = TRUE)

# Extract the expression data from each GEO dataset
exprData_gse34526 <- exprs(gse34526[[1]])  # Expression matrix for GSE34526
exprData_gse137684 <- exprs(gse137684[[1]])  # Expression matrix for GSE137684

# Extract the metadata (e.g., sample info)
phenoData_gse34526 <- pData(gse34526[[1]])
phenoData_gse137684 <- pData(gse137684[[1]])

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: generics

Attaching package: ‘generics’

The following objects are masked from ‘package:base’:

    as.difftime, as.factor, as.ordered, intersect, is.element, setdiff,
    setequal, union


Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, saveRDS, table, tapply, unique,
    unsplit, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', a

In [ ]:
%%R
library(DESeq2)

# Differential gene expression analysis for GSE34526
print("Analyzing GSE34526 with DESeq2:")

# Create DESeqDataSet object for GSE34526
# Assuming exprData_gse34526 contains count data and phenoData_gse34526 has sample information.
# You need to replace 'your_design_formula' with your actual experimental design formula.
# For example, if you have a column named 'condition' in phenoData_gse34526 indicating your groups, the formula would be '~condition'.
# If you don't have a specific design, you can use '~1' for a basic analysis, but a proper design is recommended for meaningful results.
# Replace 'your_design_formula' with the appropriate formula for your data.
dds_gse34526 <- DESeqDataSetFromMatrix(countData = exprData_gse34526,
                                       colData = phenoData_gse34526,
                                       design = ~1) # Replace ~1 with your design formula

# Run DESeq2 analysis
dds_gse34526 <- DESeq(dds_gse34526)

# Get differential expression results
# Replace 'contrast' with your actual contrast based on your design formula
# Example: results(dds_gse34526, contrast=c("condition","treatment","control"))
res_gse34526 <- results(dds_gse34526)

# Display summary of results
print(summary(res_gse34526))

# Display top genes
print(head(res_gse34526[order(res_gse34526$padj), ]))


# Differential gene expression analysis for GSE137684
print("\nAnalyzing GSE137684 with DESeq2:")

# Create DESeqDataSet object for GSE137684
# Assuming exprData_gse137684 contains count data and phenoData_gse137684 has sample information.
# You need to replace 'your_design_formula' with your actual experimental design formula.
# For example, if you have a column named 'condition' in phenoData_gse137684 indicating your groups, the formula would be '~condition'.
# If you don't have a specific design, you can use '~1' for a basic analysis, but a proper design is recommended for meaningful results.
# Replace 'your_design_formula' with the appropriate formula for your data.
dds_gse137684 <- DESeqDataSetFromMatrix(countData = exprData_gse137684,
                                        colData = phenoData_gse137684,
                                        design = ~1) # Replace ~1 with your design formula

# Run DESeq2 analysis
dds_gse137684 <- DESeq(dds_gse137684)

# Get differential expression results
# Replace 'contrast' with your actual contrast based on your design formula
# Example: results(dds_gse137684, contrast=c("condition","treatment","control"))
res_gse137684 <- results(dds_gse137684)

# Display summary of results
print(summary(res_gse137684))

# Display top genes
print(head(res_gse137684[order(res_gse137684$padj), ]))

[1] "Analyzing GSE34526 with DESeq2:"
Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘matrixStats’

The following objects are masked from ‘package:Biobase’:

    anyMissing, rowMedians


Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, 

RInterpreterError: Failed to parse and evaluate line 'library(DESeq2)\n\n# Differential gene expression analysis for GSE34526\nprint("Analyzing GSE34526 with DESeq2:")\n\n# Create DESeqDataSet object for GSE34526\n# Assuming exprData_gse34526 contains count data and phenoData_gse34526 has sample information.\n# You need to replace \'your_design_formula\' with your actual experimental design formula.\n# For example, if you have a column named \'condition\' in phenoData_gse34526 indicating your groups, the formula would be \'~condition\'.\n# If you don\'t have a specific design, you can use \'~1\' for a basic analysis, but a proper design is recommended for meaningful results.\n# Replace \'your_design_formula\' with the appropriate formula for your data.\ndds_gse34526 <- DESeqDataSetFromMatrix(countData = exprData_gse34526,\n                                       colData = phenoData_gse34526,\n                                       design = ~1) # Replace ~1 with your design formula\n\n# 

In [ ]:
%%R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 'DESeq2'
also installing the dependencies ‘formatR’, ‘lambda.r’, ‘futile.options’, ‘futile.logger’, ‘snow’, ‘BH’, ‘BiocParallel’, ‘locfit’, ‘RcppArmadillo’

trying URL 'https://cran.rstudio.com/src/contrib/formatR_1.14.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/lambda.r_1.2.4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/futile.options_1.0.1.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/futile.logger_1.4.3.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/snow_0.4-4.tar.gz'
trying URL 'https://cran.rstudio.com/src/contrib/BH_1.87.0-1.tar.gz'
trying URL 'https://bioconductor.org/packages/3.21/bioc

In [ ]:
%%R
library(limma)

# Create a design matrix
design <- model.matrix(~labels)

# Fit the linear model
fit <- lmFit(combined_exprData, design)

# Apply empirical Bayes moderation
fit <- eBayes(fit)

# Get differential expression results
deg_results <- topTable(fit, coef=2, number=Inf)

# Filter for significant genes (e.g., p-value < 0.05 and absolute logFC > 1)
significant_degs <- deg_results[deg_results$adj.P.Val < 0.05 & abs(deg_results$logFC) > 1, ]

# Display the number of significant DEGs
print(paste("Number of significant differentially expressed genes:", nrow(significant_degs)))

# Display the top 10 significant DEGs
print(head(significant_degs))


Attaching package: ‘limma’

The following object is masked from ‘package:BiocGenerics’:

    plotMA

Error in lmFit(combined_exprData, design) : 
  expression matrix has zero rows


RInterpreterError: Failed to parse and evaluate line 'library(limma)\n\n# Create a design matrix\ndesign <- model.matrix(~labels)\n\n# Fit the linear model\nfit <- lmFit(combined_exprData, design)\n\n# Apply empirical Bayes moderation\nfit <- eBayes(fit)\n\n# Get differential expression results\ndeg_results <- topTable(fit, coef=2, number=Inf)\n\n# Filter for significant genes (e.g., p-value < 0.05 and absolute logFC > 1)\nsignificant_degs <- deg_results[deg_results$adj.P.Val < 0.05 & abs(deg_results$logFC) > 1, ]\n\n# Display the number of significant DEGs\nprint(paste("Number of significant differentially expressed genes:", nrow(significant_degs)))\n\n# Display the top 10 significant DEGs\nprint(head(significant_degs))\n'.
R error message: 'Error in lmFit(combined_exprData, design) : \n  expression matrix has zero rows'
R stdout:

Attaching package: ‘limma’

The following object is masked from ‘package:BiocGenerics’:

    plotMA

Error in lmFit(combined_exprData, design) : 
  express

In [38]:
# Load necessary libraries
library(GEOquery)
library(DESeq2)
library(limma)
library(utils) # For read.table
library(dplyr) # For data manipulation

# --- Download and process GSE137684 (RNA-Seq - Count Data) ---
print("--- Processing GSE137684 (RNA-Seq) ---")

# Download supplementary files
print("Downloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# Extract the tar file
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# Define the directory containing the extracted count files
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Added comment.char = "#" to handle lines starting with #
  # Added quote = "" to prevent issues with quotes in file
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")

  # Ensure V1 and V11 exist and V11 is numeric or can be converted to numeric
  if ("V1" %in% colnames(data) && "V11" %in% colnames(data)) {
      numeric_v11 <- suppressWarnings(as.numeric(as.character(data$V11))) # Convert to character first to handle factors

      # Filter out rows where V11 is NA after conversion
      valid_rows <- !is.na(numeric_v11)

      if(any(valid_rows)) { # Check if there are any valid data rows
          data.frame(gene_id = data$V1[valid_rows], count = as.integer(round(numeric_v11[valid_rows])))
      } else {
          print(paste("Warning: No valid numeric data in V11 after skipping 6 rows in file", f, ". Returning empty data frame."))
          data.frame(gene_id = character(), count = integer()) # Return empty if no valid data
      }
  } else {
    # If V1 or V11 is not present, skip the file
    print(paste("Warning: V1 or V11 not found in file", f, ". Skipping file."))
    data.frame(gene_id = character(), count = integer()) # Return empty if columns are missing
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]

# Combine the data frames into a single matrix
if (length(count_list_gse137684) > 0) {
    # Align data frames by gene_id before combining counts
    merged_counts <- count_list_gse137684 %>% purrr::reduce(full_join, by = "gene_id")

    # Extract gene_ids and count columns
    gene_ids <- merged_counts$gene_id
    count_matrix_gse137684 <- as.matrix(merged_counts[, -1]) # Remove gene_id column

    # Replace NA values (from full_join) with 0
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0

    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- gene_ids

    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # Remove header-like rows if they were mistakenly included as genes based on gene_id content
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. No valid data found after reading and filtering.")
}


# Download the GSE137684 GEO object to get sample information
print("Downloading GSE137684 GEO object...")
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Create colData dataframe with sample information for GSE137684
print("Creating colData for GSE137684...")
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = rownames(phenoData_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
# Subset colData to match the samples actually present in the count matrix
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]

# Clean up condition levels in colData
levels(colData_gse137684$condition) <- make.names(levels(colData_gse137684$condition))


# Create a DESeqDataSet object for GSE137684
print("Creating DESeqDataSet for GSE137684...")
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column

# Run DESeq2 analysis
print("Running DESeq2 analysis for GSE137684...")
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed for GSE137684. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})

# Get differential expression results for GSE137684
print("Getting differential expression results for GSE137684...")
res_gse137684 <- NULL # Initialize to NULL
if (inherits(dds_gse137684, "DESeqDataSet")) {
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    # Define contrast based on cleaned levels (assuming "condition.Normal" is the reference if present)
    normal_level_idx <- grep("Normal", condition_levels, ignore.case = TRUE)[1] # Find index of "Normal" case-insensitively
    if (!is.na(normal_level_idx)) {
        reference_level <- condition_levels[normal_level_idx]
        other_levels <- condition_levels[condition_levels != reference_level]
        if (length(other_levels) > 0) {
             # For simplicity, compare the first non-normal group against Normal
            contrast_level <- other_levels[1]
            res_gse137684 <- results(dds_gse137684, contrast=c("condition", contrast_level, reference_level))
            print(paste("Contrast used for GSE137684:", contrast_level, "vs", reference_level))
            print("\nSummary of DESeq2 results for GSE137684:")
            print(summary(res_gse137684))
            print("\nTop genes by adjusted p-value for GSE137684:")
            print(head(res_gse137684[order(res_gse137684$padj), ]))
        } else {
            print("Error: Only 'Normal' condition found for GSE137684. Cannot perform differential expression analysis.")
        }
    } else {
        print("Warning: 'Normal' condition not found for GSE137684. Using the first two levels for contrast.")
        if (length(condition_levels) >= 2) {
             res_gse137684 <- results(dds_gse137684, contrast=c("condition", condition_levels[2], condition_levels[1]))
             print(paste("Contrast used for GSE137684:", condition_levels[2], "vs", condition_levels[1]))
             print("\nSummary of DESeq2 results for GSE137684:")
             print(summary(res_gse137684))
             print("\nTop genes by adjusted p-value for GSE137684:")
             print(head(res_gse137684[order(res_gse137684$padj), ]))
        } else {
            print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
        }
    }
  } else {
    print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
  }
} else {
  print("Error: DESeq2 analysis failed for GSE137684. dds_gse137684 is not a valid DESeqDataSet object.")
}

# Get normalized counts for GSE137684
normalized_counts_gse137684 <- NULL # Initialize to NULL
if (!is.null(dds_gse137684)) {
    normalized_counts_gse137684 <- counts(dds_gse137684, normalized=TRUE)
}


# --- Download and process GSE34526 (Microarray - Processed Data) ---
print("\n--- Processing GSE34526 (Microarray) ---")

# Download the GSE34526 GEO object
print("Downloading GSE34526 GEO object...")
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Check the data class to confirm it's not raw counts
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Create colData dataframe with sample information for GSE34526
print("Creating colData for GSE34526...")
# Identify the condition column from phenoData
condition_factor_gse34526 <- NULL # Initialize to NULL
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1.1')
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'source_name_ch1')
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1')
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Clean up level names when creating the factor
if (!is.null(condition_factor_gse34526)) {
    levels(condition_factor_gse34526) <- make.names(levels(condition_factor_gse34526))

    colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                                condition = condition_factor_gse34526)

    # Ensure row names of colData match column names of exprData
    colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]
} else {
     stop("Condition factor could not be created for GSE34526.")
}


# Use limma for analysis as data is not integer counts
print("Using limma for analysis of GSE34526...")

# Create a design matrix for limma
design_gse34526 <- model.matrix(~condition, data = colData_gse34526)

# Fit the linear model
fit_gse34526 <- lmFit(exprData_gse34526, design_gse34526)

# Apply empirical Bayes moderation
fit_gse34526 <- eBayes(fit_gse34526)

# Get differential expression results for GSE34526
print("Getting differential expression results for GSE34526...")
res_gse34526 <- NULL # Initialize to NULL
if (length(levels(colData_gse34526$condition)) >= 2) {
    # Use the column names of the design matrix to specify the contrast
    # Assuming the contrast is between the level containing "PCOS" and the level containing "normal"
    clean_levels_gse34526 <- levels(colData_gse34526$condition)
    pcos_level <- clean_levels_gse34526[grep("PCOS", clean_levels_gse34526, ignore.case = TRUE)][1]
    normal_level <- clean_levels_gse34526[grep("normal", clean_levels_gse34526, ignore.case = TRUE)][1]

    if (!is.na(pcos_level) && !is.na(normal_level)) {
        contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(pcos_level, "-", normal_level, sep=""), levels=design_gse34526)
        fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
        fit2_gse34526 <- eBayes(fit2_gse34526)
        res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")

        print(paste("Contrast used for GSE34526:", pcos_level, "vs", normal_level))
        print("\nSummary of limma results for GSE34526:")
        print(summary(res_gse34526))
        print("\nTop genes by adjusted p-value for GSE34526:")
        print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
    } else {
        print("Could not identify 'PCOS' and 'normal' conditions for GSE34526 contrast. Using the first two levels for contrast.")
         if (length(clean_levels_gse34526) >= 2) {
            contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(clean_levels_gse34526[2], "-", clean_levels_gse34526[1], sep=""), levels=design_gse34526)
            fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
            fit2_gse34526 <- eBayes(fit2_gse34526)
            res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")
            print(paste("Contrast used for GSE34526:", clean_levels_gse34526[2], "vs", clean_levels_gse34526[1]))
            print("\nSummary of limma results for GSE34526:")
            print(summary(res_gse34526))
            print("\nTop genes by adjusted p-value for GSE34526:")
            print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
         } else {
             print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
         }
    }
} else {
    print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
}

# The expression data from limma (exprData_gse34526) is already normalized.
# We will use this directly.

# --- Transfer results and normalized data to Python ---
# Transfer differential expression results
%R -o res_gse137684
%R -o res_gse34526

# Transfer normalized expression data
%R -o normalized_counts_gse137684
%R -o exprData_gse34526

# Transfer phenoData (optional, but useful for context)
%R -o phenoData_gse137684
%R -o phenoData_gse34526

print("\nR processing complete. Results and normalized data transferred to Python.")

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (ipython-input-3129309661.py, line 23)

In [ ]:
%%R
# Find common genes
common_genes <- intersect(rownames(exprData_gse34526), rownames(exprData_gse137684))

# Print the number of common genes
print(paste("Number of common genes:", length(common_genes)))

# Subset expression data to include only common genes
combined_exprData <- cbind(exprData_gse34526[common_genes, ], exprData_gse137684[common_genes, ])

# Print the dimensions of the combined expression data
print(paste("Dimensions of combined_exprData:", dim(combined_exprData)[1], "rows,", dim(combined_exprData)[2], "columns"))

# Create labels based on datasets (assuming samples in gse34526 are controls and gse137684 are cases)
labels <- c(rep(0, ncol(exprData_gse34526)), rep(1, ncol(exprData_gse137684)))

# Create a dataframe with expression data and labels
combined_data <- as.data.frame(t(combined_exprData)) # Transpose to have samples as rows and genes as columns
combined_data$label <- labels

# Display the first few rows of the combined data
print(head(combined_data))

[1] "Number of common genes: 0"
[1] "Dimensions of combined_exprData: 0 rows, 22 columns"
          label
GSM850527     0
GSM850528     0
GSM850529     0
GSM850530     0
GSM850531     0
GSM850532     0


In [ ]:
%%R
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("GEOquery")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.26.tar.gz'
Content type 'application/x-gzip' length 594489 bytes (580 KB)
downloaded 580 KB


The downloaded source packages are in
	‘/tmp/RtmpuVDt1B/downloaded_packages’
'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.21 (BiocManager 1.30.26), R 4.5.1 (2025-06-13)
Installing package(s) 'BiocVersion', 'GEOquery'
also installing the dependencies ‘matrixStats’, ‘XVector’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘SparseArray’, ‘BiocGenerics’, ‘statmod’, ‘XML’, ‘R.oo’, ‘R.methodsS3’, ‘MatrixGenerics’, ‘GenomicRanges’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘DelayedArray’, ‘Biobase’, ‘limma’, ‘rentrez’, ‘R.utils’, ‘SummarizedExperiment’, ‘S4Vectors’

trying URL 'https://cran.rst

In [ ]:
%%R
install.packages("GEOquery")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
In addition: Warning message:
package ‘GEOquery’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages 


In [40]:
### Python Code (Core ML Pipeline)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, classification_report


# Load DEG matrix (after DESeq2 output)
data = pd.read_csv("degs_expression_matrix.csv")
X = data.drop("label", axis=1) # features
y = data["label"] # 0 = Control, 1 = PCOS


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# LASSO Logistic Regression
lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
print("LASSO AUC:", roc_auc_score(y_test, lasso.predict_proba(X_test)[:,1]))


# Support Vector Machine
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict_proba(X_test)[:,1] # predict_proba returns probabilities, we need the probability of the positive class (index 1)
print("SVM AUC:", roc_auc_score(y_test, y_pred_svm))


# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict_proba(X_test)[:,1] # predict_proba returns probabilities, we need the probability of the positive class (index 1)
print("XGBoost AUC:", roc_auc_score(y_test, y_pred_xgb))

LASSO AUC: 1.0
SVM AUC: 1.0


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [10:28:06] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost AUC: 1.0


In [ ]:
### Python Code (cross-validation)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, classification_report

# Load DEG matrix (after DESeq2 output)
data = pd.read_csv("degs_expression_matrix.csv")
# Drop the 'Unnamed: 0' column if it exists, as it seems to be an artifact from saving/loading
if 'Unnamed: 0' in data.columns:
    data = data.drop('Unnamed: 0', axis=1)

X = data.drop("label", axis=1)  # features
y = data["label"]  # 0 = Control, 1 = PCOS

# Train-test split (still useful for a final evaluation, but cross-validation is better for model selection)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y) # Use stratify for balanced splits

# Initialize models
lasso = LogisticRegression(penalty='l1', solver='saga', max_iter=5000)
svm = SVC(kernel='linear', probability=True)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# Perform cross-validation
# Using StratifiedKFold to ensure folds have representative proportions of classes
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) # Using 5 splits for cross-validation

print("Cross-validation AUC scores:")

# LASSO Logistic Regression Cross-validation
lasso_cv_scores = cross_val_score(lasso, X, y, cv=cv, scoring='roc_auc')
print(f"LASSO (CV) AUC: Mean={np.mean(lasso_cv_scores):.4f}, Std={np.std(lasso_cv_scores):.4f}")

# Support Vector Machine Cross-validation
svm_cv_scores = cross_val_score(svm, X, y, cv=cv, scoring='roc_auc')
print(f"SVM (CV) AUC: Mean={np.mean(svm_cv_scores):.4f}, Std={np.std(svm_cv_scores):.4f}")

# XGBoost Cross-validation
xgb_cv_scores = cross_val_score(xgb, X, y, cv=cv, scoring='roc_auc')
print(f"XGBoost (CV) AUC: Mean={np.mean(xgb_cv_scores):.4f}, Std={np.std(xgb_cv_scores):.4f}")

# Optional: You can still evaluate on the test set for comparison
# print("\nTest set AUC scores:")
# print("LASSO AUC:", roc_auc_score(y_test, lasso.predict_proba(X_test)[:,1]))
# print("SVM AUC:", roc_auc_score(y_test, svm.predict_proba(X_test)[:,1]))
# print("XGBoost AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))

# Further steps to address overfitting and model interpretation can follow,
# such as feature selection based on cross-validation, or model interpretation
# techniques (e.g., examining LASSO coefficients or XGBoost feature importances).

Cross-validation AUC scores:
LASSO (CV) AUC: Mean=1.0000, Std=0.0000
SVM (CV) AUC: Mean=1.0000, Std=0.0000


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [09:36:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [09:36:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [09:36:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [09:36:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [09:36:18] WARNING: /w

XGBoost (CV) AUC: Mean=0.9500, Std=0.1000


In [ ]:
%%R
# Load necessary libraries
library(GEOquery)
library(DESeq2)
library(limma)
library(utils) # For read.table

# --- Download and process GSE137684 (RNA-Seq - Count Data) ---
print("--- Processing GSE137684 (RNA-Seq) ---")

# Download supplementary files
print("Downloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# Extract the tar file
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# Define the directory containing the extracted count files
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#") # Added comment.char to handle lines starting with #
  data.frame(gene_id = data$V1, count = as.integer(round(data$V11))) # Round and convert counts to integer
})

# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers)
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$"))
    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match or no files found.")
}


# Download the GSE137684 GEO object to get sample information
print("Downloading GSE137684 GEO object...")
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Create colData dataframe with sample information for GSE137684
print("Creating colData for GSE137684...")
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = rownames(phenoData_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
# Subset colData to match the samples actually read from the count files
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]

# Create a DESeqDataSet object for GSE137684
print("Creating DESeqDataSet for GSE137684...")
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column

# Run DESeq2 analysis
print("Running DESeq2 analysis for GSE137684...")
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed for GSE137684. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})

# Get differential expression results for GSE137684
print("Getting differential expression results for GSE137684...")
if (inherits(dds_gse137684, "DESeqDataSet")) {
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    # Clean up condition levels for contrast specification
    clean_condition_levels <- make.names(condition_levels)
    # Ensure the levels in colData are also cleaned
    levels(colData_gse137684$condition) <- clean_condition_levels
    # Define contrast based on cleaned levels (assuming the last level is the case, first is control)
    res_gse137684 <- results(dds_gse137684, contrast=c("condition", clean_condition_levels[length(clean_condition_levels)], clean_condition_levels[1])) # Assuming the first level is the reference
    print("\nSummary of DESeq2 results for GSE137684:")
    print(summary(res_gse137684))
    print("\nTop genes by adjusted p-value for GSE137684:")
    print(head(res_gse137684[order(res_gse137684$padj), ]))
  } else {
    print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
    res_gse137684 <- NULL
  }
} else {
  print("Error: DESeq2 analysis failed for GSE137684. dds_gse137684 is not a valid DESeqDataSet object.")
  res_gse137684 <- NULL
}

# Get normalized counts for GSE137684
if (!is.null(dds_gse137684)) {
    normalized_counts_gse137684 <- counts(dds_gse137684, normalized=TRUE)
} else {
    normalized_counts_gse137684 <- NULL
}


# --- Download and process GSE34526 (Microarray - Processed Data) ---
print("\n--- Processing GSE34526 (Microarray) ---")

# Download the GSE34526 GEO object
print("Downloading GSE34526 GEO object...")
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Check the data class to confirm it's not raw counts
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Create colData dataframe with sample information for GSE34526
print("Creating colData for GSE34526...")
# Identify the condition column from phenoData
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1.1')
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'source_name_ch1')
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1')
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Clean up level names when creating the factor
levels(condition_factor_gse34526) <- make.names(levels(condition_factor_gse34526))

colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                               condition = condition_factor_gse34526)

# Ensure row names of colData match column names of exprData
colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]


# Use limma for analysis as data is not integer counts
print("Using limma for analysis of GSE34526...")

# Create a design matrix for limma
design_gse34526 <- model.matrix(~condition, data = colData_gse34526)

# Fit the linear model
fit_gse34526 <- lmFit(exprData_gse34526, design_gse34526)

# Apply empirical Bayes moderation
fit_gse34526 <- eBayes(fit_gse34526)

# Get differential expression results for GSE34526
print("Getting differential expression results for GSE34526...")
if (length(levels(colData_gse34526$condition)) >= 2) {
    # Use the column names of the design matrix to specify the contrast
    # Assuming the contrast is between the second and first level of the condition factor
    contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(colnames(design_gse34526)[2], "-", colnames(design_gse34526)[1], sep=""), levels=design_gse34526)
    fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
    fit2_gse34526 <- eBayes(fit2_gse34526)
    res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")

    print("\nSummary of limma results for GSE34526:")
    print(summary(res_gse34526))
    print("\nTop genes by adjusted p-value for GSE34526:")
    print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))

} else {
    print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
    res_gse34526 <- NULL
}

# The expression data from limma (exprData_gse34526) is already normalized.
# We will use this directly.

# --- Transfer results and normalized data to Python ---
# Transfer differential expression results
%R -o res_gse137684
%R -o res_gse34526

# Transfer normalized expression data
%R -o normalized_counts_gse137684
%R -o exprData_gse34526

# Transfer phenoData (optional, but useful for context)
%R -o phenoData_gse137684
%R -o phenoData_gse34526

print("\nR processing complete. Results and normalized data transferred to Python.")

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [ ]:
%%R
# Load necessary libraries
library(GEOquery)
library(DESeq2)
library(limma)
library(utils) # For read.table

# --- Download and process GSE137684 (RNA-Seq - Count Data) ---
print("--- Processing GSE137684 (RNA-Seq) ---")

# Download supplementary files
print("Downloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# Extract the tar file
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# Define the directory containing the extracted count files
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
# Escaping backslashes for the pattern
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Added comment.char = "#" to handle lines starting with #
  # Added quote = "" to prevent issues with quotes in file
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")
  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  # Round and convert counts to integer
  data.frame(gene_id = data$V1, count = as.integer(round(data$V11)))
})

# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers)
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$"))
    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match or no files found.")
}


# Download the GSE137684 GEO object to get sample information
print("Downloading GSE137684 GEO object...")
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Create colData dataframe with sample information for GSE137684
print("Creating colData for GSE137684...")
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = rownames(phenoData_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
# Subset colData to match the samples actually read from the count files
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]

# Create a DESeqDataSet object for GSE137684
print("Creating DESeqDataSet for GSE137684...")
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column

# Run DESeq2 analysis
print("Running DESeq2 analysis for GSE137684...")
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed for GSE137684. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})

# Get differential expression results for GSE137684
print("Getting differential expression results for GSE137684...")
if (inherits(dds_gse137684, "DESeqDataSet")) {
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    # Clean up condition levels for contrast specification
    clean_condition_levels <- make.names(condition_levels)
    # Ensure the levels in colData are also cleaned
    levels(colData_gse137684$condition) <- clean_condition_levels
    # Define contrast based on cleaned levels (assuming "Normal" is the reference)
    # Find the index of "Normal" and other levels
    normal_idx <- which(clean_condition_levels == "condition.Normal")
    other_idx <- which(clean_condition_levels != "condition.Normal")

    if (length(normal_idx) == 1 && length(other_idx) > 0) {
        # Assuming we want to compare all other groups against "Normal"
        # For simplicity, let's pick the first non-normal group found
        contrast_level <- clean_condition_levels[other_idx[1]]
        res_gse137684 <- results(dds_gse137684, contrast=c("condition", contrast_level, clean_condition_levels[normal_idx]))
        print(paste("Contrast used for GSE137684:", contrast_level, "vs", clean_condition_levels[normal_idx]))
        print("\nSummary of DESeq2 results for GSE137684:")
        print(summary(res_gse137684))
        print("\nTop genes by adjusted p-value for GSE137684:")
        print(head(res_gse137684[order(res_gse137684$padj), ]))
    } else {
        print("Could not identify 'Normal' condition or other conditions for GSE137684 contrast.")
        res_gse137684 <- NULL
    }
  } else {
    print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
    res_gse137684 <- NULL
  }
} else {
  print("Error: DESeq2 analysis failed for GSE137684. dds_gse137684 is not a valid DESeqDataSet object.")
  res_gse137684 <- NULL
}

# Get normalized counts for GSE137684
if (!is.null(dds_gse137684)) {
    normalized_counts_gse137684 <- counts(dds_gse137684, normalized=TRUE)
} else {
    normalized_counts_gse137684 <- NULL
}


# --- Download and process GSE34526 (Microarray - Processed Data) ---
print("\n--- Processing GSE34526 (Microarray) ---")

# Download the GSE34526 GEO object
print("Downloading GSE34526 GEO object...")
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Check the data class to confirm it's not raw counts
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Create colData dataframe with sample information for GSE34526
print("Creating colData for GSE34526...")
# Identify the condition column from phenoData
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1.1')
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'source_name_ch1')
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1')
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Clean up level names when creating the factor
levels(condition_factor_gse34526) <- make.names(levels(condition_factor_gse34526))

colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                               condition = condition_factor_gse34526)

# Ensure row names of colData match column names of exprData
colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]


# Use limma for analysis as data is not integer counts
print("Using limma for analysis of GSE34526...")

# Create a design matrix for limma
design_gse34526 <- model.matrix(~condition, data = colData_gse34526)

# Fit the linear model
fit_gse34526 <- lmFit(exprData_gse34526, design_gse34526)

# Apply empirical Bayes moderation
fit_gse34526 <- eBayes(fit_gse34526)

# Get differential expression results for GSE34526
print("Getting differential expression results for GSE34526...")
if (length(levels(colData_gse34526$condition)) >= 2) {
    # Use the column names of the design matrix to specify the contrast
    # Assuming the contrast is between the level containing "PCOS" and the level containing "normal"
    clean_levels_gse34526 <- levels(colData_gse34526$condition)
    pcos_level <- clean_levels_gse34526[grep("PCOS", clean_levels_gse34526, ignore.case = TRUE)][1]
    normal_level <- clean_levels_gse34526[grep("normal", clean_levels_gse34526, ignore.case = TRUE)][1]

    if (!is.na(pcos_level) && !is.na(normal_level)) {
        contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(pcos_level, "-", normal_level, sep=""), levels=design_gse34526)
        fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
        fit2_gse34526 <- eBayes(fit2_gse34526)
        res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")

        print(paste("Contrast used for GSE34526:", pcos_level, "vs", normal_level))
        print("\nSummary of limma results for GSE34526:")
        print(summary(res_gse34526))
        print("\nTop genes by adjusted p-value for GSE34526:")
        print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
    } else {
        print("Could not identify 'PCOS' and 'normal' conditions for GSE34526 contrast.")
        res_gse34526 <- NULL
    }
} else {
    print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
    res_gse34526 <- NULL
}

# The expression data from limma (exprData_gse34526) is already normalized.
# We will use this directly.

# --- Transfer results and normalized data to Python ---
# Transfer differential expression results
%R -o res_gse137684
%R -o res_gse34526

# Transfer normalized expression data
%R -o normalized_counts_gse137684
%R -o exprData_gse34526

# Transfer phenoData (optional, but useful for context)
%R -o phenoData_gse137684
%R -o phenoData_gse34526

print("\nR processing complete. Results and normalized data transferred to Python.")

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [ ]:
%%R
# Load necessary libraries
library(GEOquery)
library(DESeq2)
library(limma)
library(utils) # For read.table

# --- Download and process GSE137684 (RNA-Seq - Count Data) ---
print("--- Processing GSE137684 (RNA-Seq) ---")

# Download supplementary files
print("Downloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# Extract the tar file
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# Define the directory containing the extracted count files
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
# Escaping backslashes for the pattern
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Added comment.char = "#" to handle lines starting with #
  # Added quote = "" to prevent issues with quotes in file
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")
  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  # Round and convert counts to integer
  # Ensure V11 exists and is not empty before accessing it
  if ("V11" %in% colnames(data)) {
    data.frame(gene_id = data$V1, count = as.integer(round(data$V11)))
  } else {
    # If V11 is not present, try to find a suitable column or skip the file
    print(paste("Warning: V11 not found in file", f, ". Skipping or adjusting column index."))
    # Attempt to find a column that might contain counts, or return an empty dataframe
    # For now, return an empty data frame to avoid errors
    data.frame(gene_id = character(), count = integer())
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]


# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match, no files found, or files are empty after reading.")
}


# Download the GSE137684 GEO object to get sample information
print("Downloading GSE137684 GEO object...")
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Create colData dataframe with sample information for GSE137684
print("Creating colData for GSE137684...")
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = rownames(phenoData_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
# Subset colData to match the samples actually read from the count files
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]

# Create a DESeqDataSet object for GSE137684
print("Creating DESeqDataSet for GSE137684...")
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column

# Run DESeq2 analysis
print("Running DESeq2 analysis for GSE137684...")
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed for GSE137684. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})

# Get differential expression results for GSE137684
print("Getting differential expression results for GSE137684...")
if (inherits(dds_gse137684, "DESeqDataSet")) {
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    # Clean up condition levels for contrast specification
    clean_condition_levels <- make.names(condition_levels)
    # Ensure the levels in colData are also cleaned
    levels(colData_gse137684$condition) <- clean_condition_levels
    # Define contrast based on cleaned levels (assuming "Normal" is the reference)
    # Find the index of "Normal" and other levels
    normal_idx <- which(clean_condition_levels == "condition.Normal")
    other_idx <- which(clean_condition_levels != "condition.Normal")

    if (length(normal_idx) == 1 && length(other_idx) > 0) {
        # Assuming we want to compare all other groups against "Normal"
        # For simplicity, let's pick the first non-normal group found
        contrast_level <- clean_condition_levels[other_idx[1]]
        res_gse137684 <- results(dds_gse137684, contrast=c("condition", contrast_level, clean_condition_levels[normal_idx]))
        print(paste("Contrast used for GSE137684:", contrast_level, "vs", clean_condition_levels[normal_idx]))
        print("\nSummary of DESeq2 results for GSE137684:")
        print(summary(res_gse1376684)) # Typo here: res_gse1376684 should be res_gse137684

        print("\nTop genes by adjusted p-value for GSE137684:")
        print(head(res_gse137684[order(res_gse137684$padj), ]))
    } else {
        print("Could not identify 'Normal' condition or other conditions for GSE137684 contrast.")
        res_gse137684 <- NULL
    }
  } else {
    print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
    res_gse137684 <- NULL
  }
} else {
  print("Error: DESeq2 analysis failed for GSE137684. dds_gse137684 is not a valid DESeqDataSet object.")
  res_gse137684 <- NULL
}

# Get normalized counts for GSE137684
if (!is.null(dds_gse137684)) {
    normalized_counts_gse137684 <- counts(dds_gse137684, normalized=TRUE)
} else {
    normalized_counts_gse137684 <- NULL
}


# --- Download and process GSE34526 (Microarray - Processed Data) ---
print("\n--- Processing GSE34526 (Microarray) ---")

# Download the GSE34526 GEO object
print("Downloading GSE34526 GEO object...")
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Check the data class to confirm it's not raw counts
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Create colData dataframe with sample information for GSE34526
print("Creating colData for GSE34526...")
# Identify the condition column from phenoData
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1.1')
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'source_name_ch1')
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1')
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Clean up level names when creating the factor
levels(condition_factor_gse34526) <- make.names(levels(condition_factor_gse34526))

colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                               condition = condition_factor_gse34526)

# Ensure row names of colData match column names of exprData
colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]


# Use limma for analysis as data is not integer counts
print("Using limma for analysis of GSE34526...")

# Create a design matrix for limma
design_gse34526 <- model.matrix(~condition, data = colData_gse34526)

# Fit the linear model
fit_gse34526 <- lmFit(exprData_gse34526, design_gse34526)

# Apply empirical Bayes moderation
fit_gse34526 <- eBayes(fit_gse34526)

# Get differential expression results for GSE34526
print("Getting differential expression results for GSE34526...")
if (length(levels(colData_gse34526$condition)) >= 2) {
    # Use the column names of the design matrix to specify the contrast
    # Assuming the contrast is between the level containing "PCOS" and the level containing "normal"
    clean_levels_gse34526 <- levels(colData_gse34526$condition)
    pcos_level <- clean_levels_gse34526[grep("PCOS", clean_levels_gse34526, ignore.case = TRUE)][1]
    normal_level <- clean_levels_gse34526[grep("normal", clean_levels_gse34526, ignore.case = TRUE)][1]

    if (!is.na(pcos_level) && !is.na(normal_level)) {
        contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(pcos_level, "-", normal_level, sep=""), levels=design_gse34526)
        fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
        fit2_gse34526 <- eBayes(fit2_gse34526)
        res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")

        print(paste("Contrast used for GSE34526:", pcos_level, "vs", normal_level))
        print("\nSummary of limma results for GSE34526:")
        print(summary(res_gse34526))
        print("\nTop genes by adjusted p-value for GSE34526:")
        print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
    } else {
        print("Could not identify 'PCOS' and 'normal' conditions for GSE34526 contrast.")
        res_gse34526 <- NULL
    }
} else {
    print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
    res_gse34526 <- NULL
}

# The expression data from limma (exprData_gse34526) is already normalized.
# We will use this directly.

# --- Transfer results and normalized data to Python ---
# Transfer differential expression results
%R -o res_gse137684
%R -o res_gse34526

# Transfer normalized expression data
%R -o normalized_counts_gse137684
%R -o exprData_gse34526

# Transfer phenoData (optional, but useful for context)
%R -o phenoData_gse137684
%R -o phenoData_gse34526

print("\nR processing complete. Results and normalized data transferred to Python.")

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR

In [ ]:
%%R
# Load necessary libraries
library(GEOquery)
library(DESeq2)
library(limma)
library(utils) # For read.table

# --- Download and process GSE137684 (RNA-Seq - Count Data) ---
print("--- Processing GSE137684 (RNA-Seq) ---")

# Download supplementary files
print("Downloading supplementary files for GSE137684...")
getGEOSuppFiles("GSE137684")

# Extract the tar file
print("Extracting GSE137684_RAW.tar...")
untar("GSE137684/GSE137684_RAW.tar", exdir = "GSE137684_extracted")

# Define the directory containing the extracted count files
count_data_dir_gse137684 <- "GSE137684_extracted"

# List all the .txt.gz files in the directory
# Escaping backslashes for the pattern
count_files_gse137684 <- list.files(count_data_dir_gse137684, pattern = "\\.txt\\.gz$", full.names = TRUE)

print("Initial R setup and file extraction for GSE137684 complete.")

[1] "--- Processing GSE137684 (RNA-Seq) ---"
[1] "Downloading supplementary files for GSE137684..."
[1] "Extracting GSE137684_RAW.tar..."
[1] "Initial R setup and file extraction for GSE137684 complete."


Using locally cached version of supplementary file(s) GSE137684 found here:
/content/GSE137684/GSE137684_RAW.tar 


In [ ]:
%%R
# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Added comment.char = "#" to handle lines starting with #
  # Added quote = "" to prevent issues with quotes in file
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")
  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  # Round and convert counts to integer
  # Ensure V11 exists and is not empty before accessing it
  if ("V11" %in% colnames(data)) {
    data.frame(gene_id = data$V1, count = as.integer(round(data$V11)))
  } else {
    # If V11 is not present, try to find a suitable column or skip the file
    print(paste("Warning: V11 not found in file", f, ". Skipping or adjusting column index."))
    # Attempt to find a column that might contain counts, or return an empty dataframe
    # For now, return an empty data frame to avoid errors
    data.frame(gene_id = character(), count = integer())
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]


# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match, no files found, or files are empty after reading.")
}

print("Count data reading and combining complete for GSE137684.")

[1] "Reading and combining count files..."
Error in round(data$V11) : non-numeric argument to mathematical function


RInterpreterError: Failed to parse and evaluate line '# Read the count data from each file and combine them into a single data frame\nprint("Reading and combining count files...")\ncount_list_gse137684 <- lapply(count_files_gse137684, function(f) {\n  # Read the file, skipping header rows and selecting the appropriate columns\n  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.\n  # Added comment.char = "#" to handle lines starting with #\n  # Added quote = "" to prevent issues with quotes in file\n  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")\n  # Assuming gene IDs are in the first column (V1) and counts are in column V11\n  # Round and convert counts to integer\n  # Ensure V11 exists and is not empty before accessing it\n  if ("V11" %in% colnames(data)) {\n    data.frame(gene_id = data$V1, count = as.integer(round(data$V11)))\n  } else {\n    # If V11 is not present, try to find a suitable column or skip th

In [ ]:
%%R
# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Added comment.char = "#" to handle lines starting with #
  # Added quote = "" to prevent issues with quotes in file
  data <- read.table(f, header = FALSE, skip = 6, fill = TRUE, comment.char = "#", quote = "")

  # --- Debugging: Inspect the structure and content of the read data ---
  print(paste("Processing file:", f))
  print("Column names after reading:")
  print(colnames(data))
  print("First few rows of data:")
  print(head(data))
  print("Class of column V11:")
  print(class(data$V11))
  # --- End Debugging ---

  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  # Round and convert counts to integer
  # Ensure V11 exists and is not empty before accessing it
  if ("V11" %in% colnames(data) && is.numeric(data$V11)) {
    data.frame(gene_id = data$V1, count = as.integer(round(data$V11)))
  } else if ("V11" %in% colnames(data) && is.character(data$V11)) {
      # If V11 is character, try converting to numeric, coercing errors to NA
      numeric_v11 <- suppressWarnings(as.numeric(data$V11))
      if (any(!is.na(numeric_v11))) { # Check if conversion was successful for any values
           data.frame(gene_id = data$V1, count = as.integer(round(numeric_v11)))
      } else {
           print(paste("Warning: V11 in file", f, "is character and cannot be converted to numeric. Skipping file or adjusting column index."))
           data.frame(gene_id = character(), count = integer()) # Return empty if conversion fails
      }
  }
  else {
    # If V11 is not present, try to find a suitable column or skip the file
    print(paste("Warning: V11 not found or is not numeric/character in file", f, ". Skipping or adjusting column index."))
    # Attempt to find a column that might contain counts, or return an empty dataframe
    # For now, return an empty data frame to avoid errors
    data.frame(gene_id = character(), count = integer())
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]


# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match, no files found, or files are empty after reading.")
}

print("Count data reading and combining complete for GSE137684.")

[1] "Reading and combining count files..."
[1] "Processing file: GSE137684_extracted/GSM4084711_1.txt.gz"
[1] "Column names after reading:"
  [1] "V1"   "V2"   "V3"   "V4"   "V5"   "V6"   "V7"   "V8"   "V9"   "V10" 
 [11] "V11"  "V12"  "V13"  "V14"  "V15"  "V16"  "V17"  "V18"  "V19"  "V20" 
 [21] "V21"  "V22"  "V23"  "V24"  "V25"  "V26"  "V27"  "V28"  "V29"  "V30" 
 [31] "V31"  "V32"  "V33"  "V34"  "V35"  "V36"  "V37"  "V38"  "V39"  "V40" 
 [41] "V41"  "V42"  "V43"  "V44"  "V45"  "V46"  "V47"  "V48"  "V49"  "V50" 
 [51] "V51"  "V52"  "V53"  "V54"  "V55"  "V56"  "V57"  "V58"  "V59"  "V60" 
 [61] "V61"  "V62"  "V63"  "V64"  "V65"  "V66"  "V67"  "V68"  "V69"  "V70" 
 [71] "V71"  "V72"  "V73"  "V74"  "V75"  "V76"  "V77"  "V78"  "V79"  "V80" 
 [81] "V81"  "V82"  "V83"  "V84"  "V85"  "V86"  "V87"  "V88"  "V89"  "V90" 
 [91] "V91"  "V92"  "V93"  "V94"  "V95"  "V96"  "V97"  "V98"  "V99"  "V100"
[101] "V101" "V102" "V103" "V104" "V105" "V106" "V107" "V108" "V109" "V110"
[111] "V111" "V112" "V11

In [ ]:
%%R
# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the file, skipping header rows and selecting the appropriate columns
  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.
  # Let's try skipping more rows to avoid header/metadata
  data <- read.table(f, header = FALSE, skip = 10, fill = TRUE, comment.char = "#", quote = "")

  # --- Debugging: Inspect the structure and content of the read data ---
  print(paste("Processing file:", f))
  print("Column names after reading:")
  print(colnames(data))
  print("First few rows of data:")
  print(head(data))
  print("Class of column V11:")
  print(class(data$V11))
  # --- End Debugging ---

  # Assuming gene IDs are in the first column (V1) and counts are in column V11
  # Round and convert counts to integer
  # Ensure V11 exists and is numeric or can be converted to numeric
  if ("V11" %in% colnames(data)) {
      numeric_v11 <- suppressWarnings(as.numeric(data$V11))
      if (any(!is.na(numeric_v11)) && is.numeric(numeric_v11)) { # Check if conversion was successful for any values and result is numeric
           data.frame(gene_id = data$V1, count = as.integer(round(numeric_v11)))
      } else {
           print(paste("Warning: V11 in file", f, "cannot be reliably converted to numeric. Skipping file or adjusting column index."))
           data.frame(gene_id = character(), count = integer()) # Return empty if conversion fails
      }
  } else {
    # If V11 is not present, try to find a suitable column or skip the file
    print(paste("Warning: V11 not found in file", f, ". Skipping or adjusting column index."))
    # Attempt to find a column that might contain counts, or return an empty dataframe
    # For now, return an empty data frame to avoid errors
    data.frame(gene_id = character(), count = integer())
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]


# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # Remove header-like rows if they were mistakenly included as genes
    header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match, no files found, or files are empty after reading.")
}

print("Count data reading and combining complete for GSE137684.")

[1] "Reading and combining count files..."
[1] "Processing file: GSE137684_extracted/GSM4084711_1.txt.gz"
[1] "Column names after reading:"
 [1] "V1"  "V2"  "V3"  "V4"  "V5"  "V6"  "V7"  "V8"  "V9"  "V10" "V11" "V12"
[13] "V13" "V14" "V15" "V16" "V17" "V18" "V19" "V20" "V21" "V22" "V23" "V24"
[25] "V25" "V26" "V27" "V28" "V29" "V30" "V31" "V32" "V33" "V34" "V35" "V36"
[37] "V37" "V38" "V39" "V40" "V41" "V42" "V43" "V44" "V45" "V46" "V47" "V48"
[49] "V49" "V50" "V51" "V52" "V53" "V54" "V55" "V56" "V57" "V58" "V59" "V60"
[61] "V61" "V62" "V63" "V64" "V65" "V66" "V67" "V68" "V69" "V70" "V71" "V72"
[1] "First few rows of data:"
    V1 V2 V3 V4
1 DATA  1  1  1
2 DATA  2  1  2
3 DATA  3  1  3
4 DATA  4  1  4
5 DATA  5  1  5
6 DATA  6  1  6
                                                                   V5
1                                                                 260
2                                                                  66
3                                             

RInterpreterError: Failed to parse and evaluate line '# Read the count data from each file and combine them into a single data frame\nprint("Reading and combining count files...")\ncount_list_gse137684 <- lapply(count_files_gse137684, function(f) {\n  # Read the file, skipping header rows and selecting the appropriate columns\n  # Based on the file structure, data starts after row 6, V1=gene ID, V11=count.\n  # Let\'s try skipping more rows to avoid header/metadata\n  data <- read.table(f, header = FALSE, skip = 10, fill = TRUE, comment.char = "#", quote = "")\n\n  # --- Debugging: Inspect the structure and content of the read data ---\n  print(paste("Processing file:", f))\n  print("Column names after reading:")\n  print(colnames(data))\n  print("First few rows of data:")\n  print(head(data))\n  print("Class of column V11:")\n  print(class(data$V11))\n  # --- End Debugging ---\n\n  # Assuming gene IDs are in the first column (V1) and counts are in column V11\n  # Round and convert cou

In [ ]:
%%R
# Read the count data from each file and combine them into a single data frame
print("Reading and combining count files...")
count_list_gse137684 <- lapply(count_files_gse137684, function(f) {
  # Read the entire file without initial skipping
  data <- read.table(f, header = FALSE, fill = TRUE, comment.char = "#", quote = "")

  # --- Debugging: Inspect the structure and content of the read data before filtering ---
  print(paste("Processing file for filtering:", f))
  print("Column names before filtering:")
  print(colnames(data))
  print("First few rows of data before filtering:")
  print(head(data))
  print("Class of column V1 after reading:")
  print(class(data$V1))
  print("Class of column V11 after reading:")
  print(class(data$V11))
  # --- End Debugging ---

  # Filter rows to keep only those that appear to be data rows
  # Assuming gene IDs in V1 do not contain '*', 'DATA', 'TYPE', or 'FEATURES'
  # Also ensure V11 is present before filtering on it
  if ("V11" %in% colnames(data)) {
      filtered_data <- data[!(data$V1 %in% c("*", "DATA", "TYPE", "FEATURES")), ]

      # Attempt to convert V11 to numeric, coercing errors to NA
      filtered_data$V11_numeric <- suppressWarnings(as.numeric(as.character(filtered_data$V11))) # Convert to character first to handle factors

      # Remove rows where V11 could not be converted to a valid number (NA)
      filtered_data <- filtered_data[!is.na(filtered_data$V11_numeric), ]

      # Check if there are any data rows left after filtering
      if(nrow(filtered_data) > 0) {
          # Select gene ID (V1) and the numeric count column
          data.frame(gene_id = filtered_data$V1, count = as.integer(round(filtered_data$V11_numeric)))
      } else {
          print(paste("Warning: No valid data rows found in file", f, "after filtering. Returning empty data frame."))
          data.frame(gene_id = character(), count = integer()) # Return empty if no valid rows
      }
  } else {
    # If V11 is not present, skip the file
    print(paste("Warning: V11 not found in file", f, ". Skipping file."))
    data.frame(gene_id = character(), count = integer()) # Return empty if V11 is missing
  }
})

# Remove empty data frames from the list
count_list_gse137684 <- count_list_gse137684[sapply(count_list_gse137684, nrow) > 0]


# Combine the data frames into a single matrix
# Ensure all files have the same gene order before combining
if (length(count_list_gse137684) > 0 && all(sapply(count_list_gse137684, function(df) identical(df$gene_id, count_list_gse137684[[1]]$gene_id)))) {
    count_matrix_gse137684 <- do.call(cbind, lapply(count_list_gse137684, function(df) df$count))
    # Replace NA values with 0 (can happen if read.table fails for some rows/cols)
    count_matrix_gse137684[is.na(count_matrix_gse137684)] <- 0
    # Set row names to gene IDs
    rownames(count_matrix_gse137684) <- count_list_gse137684[[1]]$gene_id
    # Set column names to sample names (e.g., GSM numbers) based on the files successfully read
    colnames(count_matrix_gse137684) <- gsub("\\.txt\\.gz$", "", basename(count_files_gse137684[sapply(count_list_gse137684, nrow) > 0]))

    # The previous header removal step is likely redundant after filtering, but keeping for safety
    # header_rows_to_remove <- c("DATA", "*", "TYPE", "FEATURES")
    # count_matrix_gse137684 <- count_matrix_gse137684[!rownames(count_matrix_gse137684) %in% header_rows_to_remove, ]

} else {
    stop("Error combining count data for GSE137684. Gene IDs do not match, no files found, or files are empty after reading.")
}

print("Count data reading and combining complete for GSE137684 after filtering.")

# Now, continue with the rest of the R code for GSE137684 (DESeq2 analysis) and then GSE34526 (limma analysis)
# Download the GSE137684 GEO object to get sample information
print("Downloading GSE137684 GEO object...")
gse137684_geo <- getGEO("GSE137684", GSEMatrix = TRUE)
phenoData_gse137684 <- pData(gse137684_geo[[1]])

# Create colData dataframe with sample information for GSE137684
print("Creating colData for GSE137684...")
# Use the 'condition:ch1' column as the condition
colData_gse137684 <- data.frame(row.names = rownames(phenoData_gse137684),
                                condition = factor(phenoData_gse137684$'condition:ch1'))

# Ensure the row names of colData match the column names of count_matrix_gse137684
# Subset colData to match the samples actually read from the count files
colData_gse137684 <- colData_gse137684[colnames(count_matrix_gse137684), , drop = FALSE]

# Create a DESeqDataSet object for GSE137684
print("Creating DESeqDataSet for GSE137684...")
dds_gse137684 <- DESeqDataSetFromMatrix(countData = count_matrix_gse137684,
                                        colData = colData_gse137684,
                                        design = ~condition) # Use the 'condition' column

# Run DESeq2 analysis
print("Running DESeq2 analysis for GSE137684...")
# Check if standard dispersion estimation fails and apply workaround if needed
tryCatch({
  dds_gse137684 <- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors
  dds_gse137684 <- estimateDispersions(dds_gse137684)
  dds_gse137684 <- nbinomWaldTest(dds_gse137684) # Perform Wald test
}, error = function(e) {
  if (grepl("all gene-wise dispersion estimates are within 2 orders of magnitude", e$message)) {
    message("Standard dispersion estimation failed for GSE137684. Using gene-wise estimates as final estimates.")
    dds_gse137684 <<- estimateSizeFactors(dds_gse137684) # Explicitly estimate size factors again for safety
    dds_gse137684 <<- estimateDispersionsGeneEst(dds_gse137684)
    dispersions(dds_gse137684) <<- mcols(dds_gse137684)$dispGeneEst
    dds_gse137684 <<- nbinomWaldTest(dds_gse137684) # Perform Wald test after workaround
  } else {
    stop(e) # Re-throw the error if it's not the expected one
  }
})

# Get differential expression results for GSE137684
print("Getting differential expression results for GSE137684...")
if (inherits(dds_gse137684, "DESeqDataSet")) {
  condition_levels <- levels(colData_gse137684$condition)
  if (length(condition_levels) >= 2) {
    # Clean up condition levels for contrast specification
    clean_condition_levels <- make.names(condition_levels)
    # Ensure the levels in colData are also cleaned
    levels(colData_gse137684$condition) <- clean_condition_levels
    # Define contrast based on cleaned levels (assuming "condition.Normal" is the reference if present)
    normal_level_idx <- grep("Normal", clean_condition_levels, ignore.case = TRUE)[1] # Find index of "Normal" case-insensitively
    if (!is.na(normal_level_idx)) {
        reference_level <- clean_condition_levels[normal_level_idx]
        other_levels <- clean_condition_levels[clean_condition_levels != reference_level]
        if (length(other_levels) > 0) {
             # For simplicity, compare the first non-normal group against Normal
            contrast_level <- other_levels[1]
            res_gse137684 <- results(dds_gse137684, contrast=c("condition", contrast_level, reference_level))
            print(paste("Contrast used for GSE137684:", contrast_level, "vs", reference_level))
            print("\nSummary of DESeq2 results for GSE137684:")
            print(summary(res_gse137684))

            print("\nTop genes by adjusted p-value for GSE137684:")
            print(head(res_gse137684[order(res_gse137684$padj), ]))
        } else {
            print("Error: Only 'Normal' condition found for GSE137684. Cannot perform differential expression analysis.")
            res_gse137684 <- NULL
        }
    } else {
        print("Warning: 'Normal' condition not found for GSE137684. Using the first two levels for contrast.")
        if (length(clean_condition_levels) >= 2) {
             res_gse137684 <- results(dds_gse137684, contrast=c("condition", clean_condition_levels[2], clean_condition_levels[1]))
             print(paste("Contrast used for GSE137684:", clean_condition_levels[2], "vs", clean_condition_levels[1]))
             print("\nSummary of DESeq2 results for GSE137684:")
             print(summary(res_gse137684))

             print("\nTop genes by adjusted p-value for GSE137684:")
             print(head(res_gse137684[order(res_gse137684$padj), ]))
        } else {
            print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
            res_gse137684 <- NULL
        }
    }
  } else {
    print("Error: Not enough levels in the condition variable for GSE137684 to perform differential expression analysis.")
    res_gse137684 <- NULL
  }
} else {
  print("Error: DESeq2 analysis failed for GSE137684. dds_gse137684 is not a valid DESeqDataSet object.")
  res_gse137684 <- NULL
}

# Get normalized counts for GSE137684
if (!is.null(dds_gse137684)) {
    normalized_counts_gse137684 <- counts(dds_gse137684, normalized=TRUE)
} else {
    normalized_counts_gse137684 <- NULL
}


# --- Download and process GSE34526 (Microarray - Processed Data) ---
print("\n--- Processing GSE34526 (Microarray) ---")

# Download the GSE34526 GEO object
print("Downloading GSE34526 GEO object...")
gse34526_geo <- getGEO("GSE34526", GSEMatrix = TRUE)
gse34526_geo_object <- gse34526_geo[[1]]

# Extract expression data and phenoData
exprData_gse34526 <- exprs(gse34526_geo_object)
phenoData_gse34526 <- pData(gse34526_geo_object)

# Check the data class to confirm it's not raw counts
print(paste("Class of expression data for GSE34526:", class(exprData_gse34526)))

# Create colData dataframe with sample information for GSE34526
print("Creating colData for GSE34526...")
# Identify the condition column from phenoData
if ('characteristics_ch1.1' %in% colnames(phenoData_gse34526)) { # Use 'characteristics_ch1.1' for condition
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1.1')
} else if ('source_name_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'source_name_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'source_name_ch1')
} else if ('characteristics_ch1' %in% colnames(phenoData_gse34526)) { # Fallback to 'characteristics_ch1'
  condition_factor_gse34526 <- factor(phenoData_gse34526$'characteristics_ch1')
} else {
  stop("Could not automatically identify a condition column in GSE34526 phenoData. Please manually identify and specify it.")
}

# Clean up level names when creating the factor
levels(condition_factor_gse34526) <- make.names(levels(condition_factor_gse34526))

colData_gse34526 <- data.frame(row.names = rownames(phenoData_gse34526),
                               condition = condition_factor_gse34526)

# Ensure row names of colData match column names of exprData
colData_gse34526 <- colData_gse34526[colnames(exprData_gse34526), , drop = FALSE]


# Use limma for analysis as data is not integer counts
print("Using limma for analysis of GSE34526...")

# Create a design matrix for limma
design_gse34526 <- model.matrix(~condition, data = colData_gse34526)

# Fit the linear model
fit_gse34526 <- lmFit(exprData_gse34526, design_gse34526)

# Apply empirical Bayes moderation
fit_gse34526 <- eBayes(fit_gse34526)

# Get differential expression results for GSE34526
print("Getting differential expression results for GSE34526...")
if (length(levels(colData_gse34526$condition)) >= 2) {
    # Use the column names of the design matrix to specify the contrast
    # Assuming the contrast is between the level containing "PCOS" and the level containing "normal"
    clean_levels_gse34526 <- levels(colData_gse34526$condition)
    pcos_level <- clean_levels_gse34526[grep("PCOS", clean_levels_gse34526, ignore.case = TRUE)][1]
    normal_level <- clean_levels_gse34526[grep("normal", clean_levels_gse34526, ignore.case = TRUE)][1]

    if (!is.na(pcos_level) && !is.na(normal_level)) {
        contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(pcos_level, "-", normal_level, sep=""), levels=design_gse34526)
        fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
        fit2_gse34526 <- eBayes(fit2_gse34526)
        res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")

        print(paste("Contrast used for GSE34526:", pcos_level, "vs", normal_level))
        print("\nSummary of limma results for GSE34526:")
        print(summary(res_gse34526))
        print("\nTop genes by adjusted p-value for GSE34526:")
        print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
    } else {
        print("Could not identify 'PCOS' and 'normal' conditions for GSE34526 contrast. Using the first two levels for contrast.")
         if (length(clean_levels_gse34526) >= 2) {
            contrast_matrix_gse34526 <- makeContrasts(contrasts=paste(clean_levels_gse34526[2], "-", clean_levels_gse34526[1], sep=""), levels=design_gse34526)
            fit2_gse34526 <- contrasts.fit(fit_gse34526, contrast_matrix_gse34526)
            fit2_gse34526 <- eBayes(fit2_gse34526)
            res_gse34526 <- topTable(fit2_gse34526, number=Inf, adjust.method="BH")
            print(paste("Contrast used for GSE34526:", clean_levels_gse34526[2], "vs", clean_levels_gse34526[1]))
            print("\nSummary of limma results for GSE34526:")
            print(summary(res_gse34526))
            print("\nTop genes by adjusted p-value for GSE34526:")
            print(head(res_gse34526[order(res_gse34526$adj.P.Val), ]))
         } else {
             print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
             res_gse34526 <- NULL
         }
    }
} else {
    print("Error: Not enough levels in the condition variable for GSE34526 to perform differential expression analysis using limma.")
    res_gse34526 <- NULL
}

# The expression data from limma (exprData_gse34526) is already normalized.
# We will use this directly.

# --- Transfer results and normalized data to Python ---
# Transfer differential expression results
%R -o res_gse137684
%R -o res_gse34526

# Transfer normalized expression data
%R -o normalized_counts_gse137684
%R -o exprData_gse34526

# Transfer phenoData (optional, but useful for context)
%R -o phenoData_gse137684
%R -o phenoData_gse34526

print("\nR processing complete. Results and normalized data transferred to Python.")

RParsingError: Parsing status not OK - PARSING_STATUS.PARSE_ERROR